In [3]:
#!conda create -n metrics python=3.11 ipykernel -y

In [2]:
#!activate metrics

In [4]:
#!python -m ipykernel install --user --name=metrics

In [3]:
!conda info --envs

# conda environments:
#
base                     /Users/kallol/miniconda3
metrics               *  /Users/kallol/miniconda3/envs/metrics



In [4]:
import sys
print(sys.executable)

/Users/kallol/miniconda3/envs/metrics/bin/python


#### scenario 1: when dataset is imbalanced

In [117]:
# creating random two arrays and considering them as actuals and predicted from any ML model
import random

actuals = [random.randrange(0,1) for i in range(0,1000)]+[0,0,0,0,1]   #making the data imbalanced
predicted=[random.randrange(0,1) for i in range(0,1000)]+[1,1,1,1,1]

In [118]:
from sklearn.metrics import classification_report
print(classification_report(actuals,predicted))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1004
           1       0.20      1.00      0.33         1

    accuracy                           1.00      1005
   macro avg       0.60      1.00      0.67      1005
weighted avg       1.00      1.00      1.00      1005



In [119]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(actuals,predicted))

[[1000    4]
 [   0    1]]


In [120]:
tn, fp, fn, tp = confusion_matrix(actuals,predicted).ravel()
tn, fp, fn, tp 

(1000, 4, 0, 1)

In [121]:
# calculating true positive rate - which is recall - basically saying that - out of total positive classes we are predicting - how many of them is actually positive 

tpr=tp/(tp+fn)
print(tpr)

1.0


In [122]:
fpr=fp/(fp+tn)
print(fpr)

0.00398406374501992


In [123]:
# auc-roc score when imbalance dataset is there 

from sklearn.metrics import roc_auc_score
roc_auc_score(actuals,predicted)

0.99800796812749

###### above roc-auc is very high because it considers two things- true positive rate and false positive rate, here true positive rate is very high and false positive is very low due less no of instances - eventhough the ROC is very high the model is not so great the below blog contains when to apply which metrics in binary classification scenario 
https://neptune.ai/blog/f1-score-accuracy-roc-auc-pr-auc

# 

In [127]:
# in this kind of scenarios, we should be using ppv- positive predicive value(precision)

In [128]:
# out of all class predicted as positive how many of them actually positive
ppv=tp/(tp+fp)
print(ppv)

0.2


In [53]:
from sklearn.metrics import precision_recall_curve

In [54]:
precision_recall_curve(actuals, predicted)

(array([9.95024876e-04, 2.00000000e-01, 1.00000000e+00]),
 array([1., 1., 0.]),
 array([0, 1]))

In [57]:
from sklearn.metrics import average_precision_score

In [58]:
average_precision_score(actuals, predicted)

0.2